In [1]:
%matplotlib inline

In [97]:
import torch
from fastai import datasets
import gzip
from pathlib import Path
import pickle

In [25]:
def matmul(a, b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac == br, "Matrix dimensions are not compatible for matrix multiplication."
    output = torch.zeros(ar, bc).float()
    for i in range(ar):
        for j in range(bc):
            for k in range(ac):
                output[i,j] += a[i,k] * b[k,j]   
    return output

In [26]:
a_test = torch.tensor([[1,2],[3,4]])
b_test = torch.tensor([[5,6],[7,8]])

In [27]:
a_test

tensor([[1, 2],
        [3, 4]])

In [28]:
b_test

tensor([[5, 6],
        [7, 8]])

In [29]:
%time matmul(a_test, b_test)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 593 µs


tensor([[19., 22.],
        [43., 50.]])

In [31]:
assert torch.all(matmul(a_test, b_test).eq(torch.tensor([[19,22], [43, 50]]).float())), "Results are not equal"

Check! We're looking good.

In [35]:
a_large = torch.rand(100, 100)

In [36]:
b_large = torch.rand(100, 100)

In [37]:
%time matmul(a_large, b_large)

CPU times: user 22.1 s, sys: 0 ns, total: 22.1 s
Wall time: 22.1 s


tensor([[24.6877, 26.9165, 24.7495,  ..., 25.8846, 25.9685, 24.7354],
        [26.7146, 28.1214, 28.5030,  ..., 29.0957, 28.0208, 26.2065],
        [22.5379, 24.6427, 22.7637,  ..., 23.4620, 23.9824, 22.7329],
        ...,
        [24.3042, 28.0166, 25.5609,  ..., 25.5415, 27.5984, 24.7170],
        [24.7097, 26.5033, 25.4868,  ..., 25.5333, 25.8876, 25.3801],
        [24.3454, 26.5077, 25.1758,  ..., 26.7148, 24.7056, 24.4630]])

In [58]:
def matmul2(a, b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac == br, "Matrix dimensions are not compatible for matrix multiplication."
    output = torch.zeros(ar, bc).float()
    for i in range(ar):
        for j in range(bc):
            output[i,j] = (a[i] * b[:,j]).sum()
    return output

In [59]:
assert torch.all(matmul2(a_test, b_test).eq(torch.tensor([[19,22], [43, 50]]).float()))

In [60]:
%time matmul2(a_large, b_large)

CPU times: user 240 ms, sys: 0 ns, total: 240 ms
Wall time: 241 ms


tensor([[24.6877, 26.9165, 24.7495,  ..., 25.8846, 25.9685, 24.7354],
        [26.7146, 28.1214, 28.5030,  ..., 29.0957, 28.0208, 26.2065],
        [22.5379, 24.6427, 22.7637,  ..., 23.4620, 23.9824, 22.7329],
        ...,
        [24.3042, 28.0166, 25.5609,  ..., 25.5415, 27.5984, 24.7170],
        [24.7097, 26.5033, 25.4868,  ..., 25.5333, 25.8876, 25.3801],
        [24.3454, 26.5077, 25.1758,  ..., 26.7148, 24.7056, 24.4630]])

Much, much faster.

In [61]:
def matmul3(a, b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac == br, "Matrix dimensions are not compatible for matrix multiplication."
    return torch.einsum('ik,kj->ij', a, b).float()

In [62]:
assert torch.all(matmul3(a_test, b_test).eq(torch.tensor([[19,22], [43, 50]]).float()))

In [63]:
%time matmul3(a_large, b_large)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 13.1 ms


tensor([[24.6877, 26.9165, 24.7495,  ..., 25.8846, 25.9685, 24.7354],
        [26.7146, 28.1214, 28.5030,  ..., 29.0957, 28.0208, 26.2065],
        [22.5379, 24.6427, 22.7637,  ..., 23.4620, 23.9824, 22.7329],
        ...,
        [24.3042, 28.0166, 25.5609,  ..., 25.5415, 27.5984, 24.7170],
        [24.7097, 26.5033, 25.4868,  ..., 25.5333, 25.8876, 25.3801],
        [24.3454, 26.5077, 25.1758,  ..., 26.7148, 24.7056, 24.4630]])

# Broadcasting Exploration

In [65]:
x = torch.tensor(5)

In [66]:
y = torch.tensor([1,2,3])

In [67]:
x.expand_as(y)

tensor([5, 5, 5])

In [68]:
x.shape, x.expand_as(y).shape

(torch.Size([]), torch.Size([3]))

In [72]:
x = torch.rand(3,4)

In [73]:
y = torch.tensor([1,2,3,4])

In [74]:
x.shape, y.shape

(torch.Size([3, 4]), torch.Size([4]))

In [76]:
y.expand_as(x).shape

torch.Size([3, 4])

In [77]:
y

tensor([1, 2, 3, 4])

In [78]:
x = torch.rand(3,4)

In [79]:
y = torch.tensor([1,2,3])

In [81]:
y.expand_as(x)

RuntimeError: The expanded size of the tensor (4) must match the existing size (3) at non-singleton dimension 1.  Target sizes: [3, 4].  Tensor sizes: [3]

In [99]:
MNIST_URL='http://deeplearning.net/data/mnist/mnist.pkl'
path = datasets.download_data(MNIST_URL, ext='.gz')

In [100]:
with gzip.open(path, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [101]:
x_train.shape

(50000, 784)

In [102]:
y_train.shape

(50000,)

In [103]:
x_valid.shape

(10000, 784)

In [104]:
y_valid.shape

(10000,)

In [105]:
type(x_train)

numpy.ndarray